In [ ]:
from dictdiffer import diff
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

new_url = "http://127.0.0.1:6006/graphql"
old_url = "http://127.0.0.1:6008/graphql"

client_new_url = Client(
    transport=RequestsHTTPTransport(url=new_url, timeout=1),
    fetch_schema_from_transport=True,
)
client_old_url = Client(
    transport=RequestsHTTPTransport(url=old_url, timeout=1),
    fetch_schema_from_transport=True,
)

In [ ]:
spans_query = gql(
    """query SpansQuery($projectId: GlobalID!, $after: String = null, $before: String = null, $first: Int = null, $last: Int = null) {
  node(id: $projectId) {
    ...on Project {
      spans(after: $after, before: $before, first: $first, last: $last) {
        edges {
          node {
            name
          }
        }
      }
    }
  }
}"""
)

In [ ]:
list(
    diff(
        client_old_url.execute(
            spans_query, variable_values={"projectId": "UHJvamVjdDow", "first": 10}
        ),
        client_new_url.execute(
            spans_query, variable_values={"projectId": "UHJvamVjdDox", "first": 10}
        ),
        tolerance=0.0001,
    )
)